In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from collections import Counter
import os
from argparse import Namespace
import collections

In [2]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


##Лаб_4. Генерация текстов на основе LSTM


Задание 1. Загрузите текст из произведений Ницше ('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt').
Выведете следующее:
* А) длину всего корпуса;
* Б) количество предложений;
* В) сколько всего символов используется?


In [3]:
f = open(r'/content/drive/MyDrive/Colab Notebooks/тексты/4_LSTM/nietzsche.txt', 'r').read().lower()
type(f)

str

In [4]:
print(f"Symbols: {len(f)}\nUnique symbols: {len(set(f))}",
      f"\n\nWords: {len(f.split(' '))}\nTokenized words: {len(word_tokenize(f))}",
      f"\nUnique words: {len(set(f.split(' ')))}\nUnique tokenized words: {len(set(word_tokenize(f)))}",
      f"\n\nSentences : {len(f.split('.'))}\nTokenized sentences: {len(sent_tokenize(f))}")

Symbols: 600893
Unique symbols: 57 

Words: 91209
Tokenized words: 117653 
Unique words: 22183
Unique tokenized words: 11012 

Sentences : 2788
Tokenized sentences: 2864


In [4]:
class Vocab:
    def __init__(self, data):
        self.data = data
        self.idx_to_token = dict(enumerate(self.data))
        self.token_to_idx = {char : idx for idx, char in self.idx_to_token.items()}
        self.vocab_len = len(self.token_to_idx)

vocab_words = Vocab(set(word_tokenize(f)))
vocab_chars = Vocab(set(f))

In [6]:
print(vocab_chars.idx_to_token)

{0: 'j', 1: 'a', 2: 'c', 3: 'b', 4: 'm', 5: '8', 6: ',', 7: '-', 8: '0', 9: '.', 10: 'r', 11: '3', 12: 'k', 13: '1', 14: 'o', 15: ']', 16: 'ë', 17: ':', 18: 'l', 19: '6', 20: '(', 21: '[', 22: '?', 23: '!', 24: 'æ', 25: ' ', 26: 'y', 27: 'i', 28: 'e', 29: 'v', 30: 't', 31: 'u', 32: '2', 33: 'd', 34: 'é', 35: 'z', 36: ')', 37: '"', 38: '\n', 39: 'g', 40: ';', 41: 'w', 42: 'x', 43: 'ä', 44: '=', 45: '4', 46: 's', 47: 'h', 48: 'q', 49: '9', 50: "'", 51: '5', 52: 'f', 53: 'p', 54: '_', 55: '7', 56: 'n'}


Задание 2. Сократите текст наполовину избыточными последовательностями символов maxlen
* определить словарь vocab
* разбить на проследовательности по 40 токенов с шагом 3
* векторизовать последовательности



In [ ]:
"""
по буквам - передавать строку
по словам - передавать список слов
"""

def sequence(text, vocab, maxlen = 40, step = 3):
    sentences = []
    next_tokens = []
    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_tokens.append(text[i + maxlen])

    """
    для каждого предложения, для каждого слова в предложении
    x : в положении [индекс предложения, индекс токена в предложении, индекс этого же токена из словаря] = 1
    y : в положении [индекс предложения, индекс следующего токена из словаря] = 1
    """

    print('Vectorization...')
    x = np.zeros((len(sentences), maxlen, len(vocab)))
    y = np.zeros((len(sentences), len(vocab)))
    for s, sent in enumerate(sentences):
        for t, token in enumerate(sent):
            x[s, t, vocab[token]] = 1
        y[s, vocab[next_tokens[s]]] = 1
    return x, y

In [ ]:
x, y = sequence(word_tokenize(f), vocab_words.token_to_idx)

Vectorization...


In [ ]:
x.shape

(39205, 40, 11012)

Задание 3. Создайте модель LSTM для генерации текста
* А) Напишите вспомогательную функцию для выборки индекса из массива вероятностей
* Б) Напишите функцию, которая будет вызываться в конце каждой эпохи и печатать сгенерированный текст
* В) Запустите модель на обучение
Имейте ввиду, что требуется не менее 20 эпох, прежде чем сгенерированный текст начнет звучать связно. Рекомендуется запускать этот скрипт на графическом процессоре, так как рекуррентные сети требуют довольно больших вычислительных затрат.
* Г) Проверьте работу модели в онлайн режиме.
Для примера смотрите файл lstm_генерация.ipynb


Задание 4. Создайте самостоятельно генерацию текста для РУССКОЯЗЫЧНОГО НАБОРА глав Wikibooks.
Полный текст Wikibooks содержит более 270000 глав на 12 языках https://www.kaggle.com/datasets/dhruvildave/wikibooks-dataset/data


In [5]:
is_cuda = False

if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [6]:
Flags = collections.namedtuple('Flags', 'text seq_size batch_size embedding_size lstm_size gradients_norm initial_words predict_top_k checkpoint_path learning_rate')
# print with_class._fields

flags = Flags(
    text=f,
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['woman', 'do'],
    predict_top_k=5,
    checkpoint_path='checkpoint',
    learning_rate = 0.01
)

In [8]:
def get_data_from_file(text, batch_size, seq_size):
    text = text.split()

    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {k: w for k, w in enumerate(sorted_vocab)}
    vocab_to_int = {w: k for k, w in int_to_vocab.items()}
    n_vocab = len(int_to_vocab)

    print('Vocabulary size ', n_vocab)

    int_text = [vocab_to_int[w] for w in text]
    num_batches = int(len(int_text) / (seq_size * batch_size))

    print(f'num_batches: {num_batches}')

    in_text = int_text[:num_batches * batch_size * seq_size]
    out_text = np.zeros_like(in_text)
    out_text[:-1] = in_text[1:]
    out_text[-1] = in_text[0]
    in_text = np.reshape(in_text, (batch_size, -1))
    out_text = np.reshape(out_text, (batch_size, -1))

    return int_to_vocab, vocab_to_int, n_vocab, in_text, out_text

In [9]:
def get_batches(in_text, out_text, batch_size, seq_size):
    num_batches = np.prod(in_text.shape) // (seq_size * batch_size)
    for i in range(0, num_batches * seq_size, seq_size):
        yield in_text[:, i:i+seq_size], out_text[:, i:i+seq_size]

In [10]:
class RNNModule(nn.Module):
    def __init__(self, n_vocab, seq_size, embedding_size, lstm_size):
        super(RNNModule, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size

        self.embedding = nn.Embedding(n_vocab, embedding_size)
        self.lstm = nn.LSTM(embedding_size,
                            lstm_size,
                            batch_first=True)
        self.dense = nn.Linear(lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.dense(output)
        return logits, state

    def zero_state(self, batch_size):
        return (torch.zeros(1, batch_size, self.lstm_size),
                torch.zeros(1, batch_size, self.lstm_size))

In [11]:
def predict(device, model, words, n_vocab, vocab_to_int, int_to_vocab, top_k=5):
    model.eval()

    state_h, state_c = model.zero_state(1)
    state_h = state_h.to(device)
    state_c = state_c.to(device)

    for w in (w for wl in words for w in wl):
        ix = torch.tensor([[vocab_to_int[w]]], dtype=torch.long).to(device)
        output, (state_h, state_c) = model(ix, (state_h, state_c))

    _, top_ix = torch.topk(output[0], k=top_k)
    choices = top_ix.tolist()
    choice = np.random.choice(choices[0])

    words_new = list()
    words_new.append(int_to_vocab[choice])

    for _ in range(50):
        ix = torch.tensor([[choice]], dtype=torch.long).to(device)
        output, (state_h, state_c) = model(ix, (state_h, state_c))

        _, top_ix = torch.topk(output[0], k=top_k) # A namedtuple of (values, indices) is returned
        choices = top_ix.tolist()
        choice = np.random.choice(choices[0])
        words_new.append(int_to_vocab[choice])

    words.append(words_new)

    print('\n\n'.join(' '.join(s) for s in words))

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
    flags.text, flags.batch_size, flags.seq_size)

iteration = 20000

model = RNNModule(n_vocab, flags.seq_size, flags.embedding_size, flags.lstm_size)
model = model.to(device)
model.load_state_dict(torch.load(f"/content/drive/MyDrive/Colab Notebooks/тексты/4_LSTM/checkpoint_pt/model-20000.pth"))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=flags.learning_rate)

Vocabulary size  17682
num_batches: 193


In [ ]:
from tqdm import tqdm

n_epochs = 3

for e in tqdm(range(n_epochs)):
    batches = get_batches(in_text, out_text, flags.batch_size, flags.seq_size)
    state_h, state_c = model.zero_state(flags.batch_size)

    state_h = state_h.to(device)
    state_c = state_c.to(device)

    for x, y in batches:
#         print(type(x), x.shape, x)
        iteration += 1

        model.train() # Tell it we are in training mode

        optimizer.zero_grad() # Reset all gradients

        x = torch.tensor(x, dtype=torch.long).to(device)
        y = torch.tensor(y, dtype=torch.long).to(device)

        logits, (state_h, state_c) = model(x, (state_h, state_c))
        loss = criterion(logits.transpose(1, 2), y)

        state_h = state_h.detach()
        state_c = state_c.detach()

        loss_value = loss.item()

        loss.backward() # Perform back-propagation

        _ = torch.nn.utils.clip_grad_norm_(
            model.parameters(), flags.gradients_norm)

        optimizer.step()

        if iteration % 50 == 0:
            print(f'Epoch: {e}/{n_epochs}\nIteration: {iteration}\nLoss: {loss_value}')

        if iteration % 1000 == 0:
            predict(device, model, [flags.initial_words], n_vocab, vocab_to_int, int_to_vocab, top_k=5)

            torch.save(model.state_dict(),
                       f'/content/drive/MyDrive/Colab Notebooks/тексты/4_LSTM/checkpoint_pt/model-{iteration}.pth')

In [19]:
karenina = open(r"/content/drive/MyDrive/Colab Notebooks/тексты/4_LSTM/AnnaKarenina__.txt", "r").read().lower()

In [20]:
Flags = collections.namedtuple('Flags', 'text seq_size batch_size embedding_size lstm_size gradients_norm initial_words predict_top_k checkpoint_path learning_rate')
flags2 = Flags(
    text=karenina,
    seq_size=32,
    batch_size=16,
    embedding_size=64,
    lstm_size=64,
    gradients_norm=5,
    initial_words=['я', 'очень'],
    predict_top_k=5,
    checkpoint_path='checkpoint',
    learning_rate = 0.01
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
    flags2.text, flags2.batch_size, flags2.seq_size)

model = RNNModule(n_vocab, flags2.seq_size, flags2.embedding_size, flags2.lstm_size)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=flags2.learning_rate)

iteration = 0

Vocabulary size  53301
num_batches: 576


In [ ]:
n_epochs = 200

for e in range(n_epochs):
    batches = get_batches(in_text, out_text, flags2.batch_size, flags2.seq_size)
    state_h, state_c = model.zero_state(flags2.batch_size)

    state_h = state_h.to(device)
    state_c = state_c.to(device)

    for x, y in batches:
#         print(type(x), x.shape, x)
        iteration += 1

        model.train() # Tell it we are in training mode

        optimizer.zero_grad() # Reset all gradients

        x = torch.tensor(x, dtype=torch.long).to(device)
        y = torch.tensor(y, dtype=torch.long).to(device)

        logits, (state_h, state_c) = model(x, (state_h, state_c))

        loss = criterion(logits.transpose(1, 2), y)

        state_h = state_h.detach()
        state_c = state_c.detach()

        loss_value = loss.item()

        loss.backward() # Perform back-propagation

        # gradient clipping:
        _ = torch.nn.utils.clip_grad_norm_(
            model.parameters(), flags2.gradients_norm)

        optimizer.step()

        if iteration % 50 == 0:
            print(f'Epoch: {e}/{n_epochs} Iteration: {iteration} Loss: {loss_value}')

        if iteration % 1000 == 0:
            predict(device, model, [flags2.initial_words], n_vocab, vocab_to_int, int_to_vocab)

            torch.save(model.state_dict(),
                       f'/content/drive/MyDrive/Colab Notebooks/тексты/4_LSTM/checkpoint_pt/model_ru-{iteration}.pth')

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
    flags2.text, flags2.batch_size, flags2.seq_size)

iteration = 6000

model = RNNModule(n_vocab, flags2.seq_size, flags2.embedding_size, flags2.lstm_size)
model = model.to(device)
model.load_state_dict(torch.load(f"/content/drive/MyDrive/Colab Notebooks/тексты/4_LSTM/checkpoint_pt/model_ru-{iteration}.pth"))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=flags2.learning_rate)

predict(device, model, [['помоги', 'мне']], n_vocab, vocab_to_int, int_to_vocab)

Vocabulary size  53301
num_batches: 576
помоги мне

только и я скажу ей?» и я знаю, не буду, что вы знаете, – сказала княгиня. и опять туда показалось, – что я могу отдать его чем-нибудь?» и я кое-как сказать… я делаюсь воли наступило мертвое, в 70-е годы: обнятою огромной руки. она была не одна, возможна и сестрой к нему


In [22]:
n_epochs = 20

for e in range(n_epochs):
    batches = get_batches(in_text, out_text, flags2.batch_size, flags2.seq_size)
    state_h, state_c = model.zero_state(flags2.batch_size)

    state_h = state_h.to(device)
    state_c = state_c.to(device)

    for x, y in batches:
#         print(type(x), x.shape, x)
        iteration += 1

        model.train() # Tell it we are in training mode

        optimizer.zero_grad() # Reset all gradients

        x = torch.tensor(x, dtype=torch.long).to(device)
        y = torch.tensor(y, dtype=torch.long).to(device)

        logits, (state_h, state_c) = model(x, (state_h, state_c))

        loss = criterion(logits.transpose(1, 2), y)

        state_h = state_h.detach()
        state_c = state_c.detach()

        loss_value = loss.item()

        loss.backward() # Perform back-propagation

        # gradient clipping:
        _ = torch.nn.utils.clip_grad_norm_(
            model.parameters(), flags2.gradients_norm)

        optimizer.step()

        if iteration % 50 == 0:
            print(f'Epoch: {e}/{n_epochs} Iteration: {iteration} Loss: {loss_value}')

        if iteration % 1000 == 0:
            predict(device, model, [flags2.initial_words], n_vocab, vocab_to_int, int_to_vocab)

            torch.save(model.state_dict(),
                       f'/content/drive/MyDrive/Colab Notebooks/тексты/4_LSTM/checkpoint_pt/model_ru-{iteration}.pth')

Epoch: 0/20 Iteration: 6050 Loss: 3.493364095687866
Epoch: 0/20 Iteration: 6100 Loss: 3.705981731414795
Epoch: 0/20 Iteration: 6150 Loss: 3.8550615310668945
Epoch: 0/20 Iteration: 6200 Loss: 4.063878536224365
Epoch: 0/20 Iteration: 6250 Loss: 4.2456254959106445
Epoch: 0/20 Iteration: 6300 Loss: 3.994547128677368
Epoch: 0/20 Iteration: 6350 Loss: 3.6236371994018555
Epoch: 0/20 Iteration: 6400 Loss: 4.201777935028076
Epoch: 0/20 Iteration: 6450 Loss: 5.041351318359375
Epoch: 0/20 Iteration: 6500 Loss: 4.99310302734375
Epoch: 0/20 Iteration: 6550 Loss: 5.000590801239014
Epoch: 1/20 Iteration: 6600 Loss: 3.295138120651245
Epoch: 1/20 Iteration: 6650 Loss: 2.9929749965667725
Epoch: 1/20 Iteration: 6700 Loss: 3.3319642543792725
Epoch: 1/20 Iteration: 6750 Loss: 3.2159202098846436
Epoch: 1/20 Iteration: 6800 Loss: 3.300675392150879
Epoch: 1/20 Iteration: 6850 Loss: 3.330460786819458
Epoch: 1/20 Iteration: 6900 Loss: 3.606283187866211
Epoch: 1/20 Iteration: 6950 Loss: 3.0831797122955322
Epoch:

KeyboardInterrupt: ignored

In [ ]:
print(logits.transpose(1, 2)[0].shape, y[0].shape)

torch.Size([53301, 32]) torch.Size([32])


In [ ]:
print(logits.transpose(1, 2)[0, 0])

tensor([-3.9834, -5.4743, -4.1930, -4.5456, -6.2130, -5.5867, -6.6796, -4.2893,
        -3.9535, -3.0015, -5.0683, -3.1591, -4.4779, -2.1410, -3.1516, -3.7769,
        -3.6955, -2.4978, -5.4208, -6.8305, -7.6931, -3.6254, -6.8509, -5.9198,
        -3.5860, -2.4918, -2.1399, -6.2458, -8.5704, -3.4032, -1.6102, -9.4341],
       grad_fn=<SelectBackward0>)


In [ ]:
print(logits[0])

tensor([[ -3.9834, -10.9878,  -5.0249,  ..., -17.8426, -17.7655, -17.8216],
        [ -5.4743,  -4.7344,  -4.4455,  ..., -24.1769, -24.7833, -24.4461],
        [ -4.1930,  -3.5709,  -4.1695,  ..., -15.3681, -15.6860, -15.5729],
        ...,
        [ -3.4032,  -7.6013, -12.5809,  ..., -29.7955, -30.3856, -29.8458],
        [ -1.6102,  -4.7668,  -1.5558,  ..., -18.9417, -19.5016, -19.0712],
        [ -9.4341,  -8.5210,  -8.1673,  ..., -19.7130, -20.1074, -19.7240]],
       grad_fn=<SelectBackward0>)


In [ ]:
y[0]

tensor([ 1509,     3,   155,     0,    27,  2661,     4, 21380,     1,  2206,
          233,    11,     8,     2,   933,   468,     1,    19,   105,   907,
        12235,     0,   123,    23,     5,   769,    20,  3750, 12236,     0,
           20,    61])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
int_to_vocab, vocab_to_int, n_vocab, in_text, out_text = get_data_from_file(
    flags.text, flags.batch_size, flags.seq_size)

iteration = 9600

model = RNNModule(n_vocab, flags.seq_size, flags.embedding_size, flags.lstm_size)
model = model.to(device)
model.load_state_dict(torch.load(f"/content/drive/MyDrive/Colab Notebooks/тексты/4_LSTM/checkpoint_pt/model-{iteration}.pth"))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=flags.learning_rate)

predict(device, model, [['help', 'me']], n_vocab, vocab_to_int, int_to_vocab)

Vocabulary size  17682
num_batches: 193
help me

herself, and so doing, the same as an comprehensiveness would we clearly in the drawer with which we said, for as galiani still survives. after the entire modern men, of their masters and revolt at the last century, when precisely this worthy love, be courses of thought, they set men and
